In [51]:
import pandas as pd
import geopandas as gpd
import glob
import os
# Operations on geometries
import shapely
import numpy as np
import math
import xarray as xr
import xarray
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn import linear_model
import statsmodels.api as sm
import seaborn as sns
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from tqdm import tqdm_notebook as tqdm

In [52]:
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures (change the path!)
PROJECT_ROOT_DIR = "/Users/noeliaotero/Documents/CAS_ML/data_test/"
CHAPTER_ID = "ML"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [113]:
def readcsv(csvfiles, DATE_START, DATE_END, l_vars):
    """"Read CSV files located in DIR_DATA
        Select the time period """
    dataframes = []  # a list to hold all the individual pandas DataFrames
    for ifile in range(0,len(l_files)):
        df = pd.read_csv(l_files[ifile])
        print(l_vars[ifile])
        if (l_vars[ifile] == 'precip'):
            df_time = pd.to_datetime({
                'year': df.year,
                'month': df.month,
                'day': df.day})
            df.insert(0, "date", df_time, True)
            # select the total column (CH)
            df= df[['date','reg_tot']]
        else:
            df = convertUnits(df, l_vars[ifile])
            
        df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d').dt.date
        df = df[(df['date'] >= DATE_START) & (df['date'] <= DATE_END)]
        
        dataframes.append(df)
        
    return(dataframes)

In [114]:
def convertUnits(df, mvar):
    """Convert each to the corresponding units"""
    
    if(mvar == 'T2MMEAN'):
        df[mvar] = df[mvar] -273.15
    elif(mvar == 'Z'):
        df[df.columns[1:6]] = df[df.columns[1:6]]/G
    elif (mvar == 'MSL'):
        df[mvar] = df[mvar]/100
        
    return(df)
    

In [115]:
# load the data
G = 9.80665
DIR_DATA = "/Users/noeliaotero/Documents/CAS_ML/data_test/"
#DATE_START = '1979-01-01'
DATE_START = pd.to_datetime('1979-01-01').date()
#DATE_END = '2020-12-31'
DATE_END = pd.to_datetime('2020-12-31').date()
l_files  = glob.glob(os.path.join(DIR_DATA, '*.csv'))
l_vars  = ['Z', 'MSL', 'T2MMEAN', 'precip']

In [116]:
df = pd.read_csv(l_files[1])
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d').dt.date
#df = df[(df['date'] >= DATE_START) & (df['date'] <= DATE_END)]
df = df[(df['date'] >= DATE_START) & (df['date'] <= DATE_END)]

In [117]:
l_all = readcsv(l_files, DATE_START, DATE_END, l_vars)

Z
MSL
T2MMEAN
precip


In [122]:
pd.concat(l_all, axis=1)

,date,1000,850,700,500,300,date,MSL,date,T2MMEAN,date,reg_tot
0,1979-01-01,68.369168,1340.679233,2808.103073,5257.976679,8709.221804,1979-01-01,1009.0970,1979-01-01,-9.09492,1979-01-01,1.611
1,1979-01-02,199.460040,1441.089057,2869.504877,5237.282864,8605.786890,1979-01-02,1026.4789,1979-01-02,-16.21680,1979-01-02,3.927
2,1979-01-03,187.244992,1449.997094,2913.765761,5340.855440,8780.448981,1979-01-03,1024.0898,1979-01-03,-13.92896,1979-01-03,0.613
3,1979-01-04,72.769284,1356.885379,2861.477875,5356.089286,8819.888545,1979-01-04,1009.1322,1979-01-04,-13.54580,1979-01-04,0.723
4,1979-01-05,127.660414,1424.216323,2942.889264,5446.512316,8893.579357,1979-01-05,1015.9122,1979-01-05,-12.99600,1979-01-05,0.122
...,...,...,...,...,...,...,...,...,...,...,...,...
15336,2020-12-27,35.902183,1343.210576,2873.518276,5427.833868,8987.684887,NaN,NaN,2020-12-27,-6.79212,2020-12-27,2.733
15337,2020-12-28,-112.112893,1195.249754,2710.293015,5206.820372,8642.969821,NaN,NaN,2020-12-28,-3.55503,2020-12-28,4.433
15338,2020-12-29,-31.685609,1277.511587,2788.747432,5265.210852,8662.936885,NaN,NaN,2020-12-29,-3.35328,2020-12-29,2.144
15339,2020-12-30,58.711226,1360.830151,2863.695554,5325.833593,8721.543544,NaN,NaN,2020-12-30,-5.37660,2020-12-30,0.549


In [52]:
# Create exceedances of precipitation
def prec_ex(dd):
    


precip_p95 = precip.copy()
precip_p99 = precip.copy()

for key, ts in precip.iteritems():
    if key in ['year', 'month', 'day']: continue
    precip_p95[key] = ts > ts.quantile(0.95)
    precip_p99[key] = ts > ts.quantile(0.99)

In [ ]:
def split_data(df_hp, yy_split, attributes):
    train_dataset = df_hp[pd.to_datetime(df_hp["date"]).dt.year>=yy_split]
    test_dataset = df_hp[pd.to_datetime(df_hp["date"]).dt.year<=yy_split] # for reconstruction
    # extract the dates for each datasets
    train_dates = train_dataset['date']
    test_dates = test_dataset['date']
    # extract labels
    train_labels = train_dataset["generation"].copy()
    test_labels = test_dataset["generation"].copy()
    # extract predictors
    train_dataset = train_dataset[attributes]
    test_dataset = test_dataset[attributes]

    return(train_dataset, train_labels, test_dataset, test_labels, train_dates, test_dates)

In [ ]:
def prepareData(train_dataset):
    
    num_attribs = list(train_dataset)
    
    num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

    full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
    ])

    df_prepared = full_pipeline.fit_transform(train_dataset)
    return(df_prepared)
    